### Задание по проекту. 

Для его выполнения вам понадобится собранная коллекция документов и функция, составляющая обратный индекс по словам в коллекции.

Напишите функцию (или несколько отдельных логичный функций), которая по запросу  Q=q1,...,gnQ=q1,...,gn  и коллекции  DD  сортирует выдачу подходящих документов. Будем считать документ подходящим, если он содержит хотя бы одно слово из запроса (из которого удалены стоп-слова). В качестве метрики используйте Okapi BM25.
Для проверки работы функции на вашем корпусе используйте запрос каникулы на новый год и рождество. Выведите ссылки в ipynb на первые десять докуменов в отсортированной выдаче(как во втором семинаре с помощью IPython.display) и их оценку BM25. Напомню, что ссылки на документы хрянятся в самих доках под тэгом @url.

В этом случае можно использовать формулу *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)). Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где $f(q_i,D)$ - частота слова (TF) $q_i$ в документе $D$, $|D|$ - длина документа (количество слов в нём), а *avgdl* — средняя длина документа в коллекции. 
$$$$
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75.
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
где $N$ - общее количество документов в коллекции, а  $n(q_i)$ — количество документов, содержащих $q_i$. 

In [1]:
from math import log
from collections import Counter,  defaultdict
import glob
from pymystem3 import Mystem
import re
import operator
import json
import tqdm

In [2]:
'''n - number of docs with a certain word (number of indexes)
qf - word frequency in the doc
N - number of documents in the collection
dl - doc length(number of words in the doc)
avdl - medium length of doc in the collection
'''

def score_BM25(n, qf, N, dl, avdl):
    K = compute_K(dl, avdl)
    IDF = log((N - n + 0.5) / (n + 0.5))
    frac = ((k1 + 1) * qf) / (K + qf)
    
    return IDF * frac
    

def compute_K(dl, avdl):
    return k1 * ((1-b) + b * (float(dl)/float(avdl))) 


In [3]:
def BM25_foralldocs(query, list_of_docs, frequency, index_dict, BM25_for_all, stop_words):
    
    avdl = 0
    total_dl = 0
    
    for doc in list_of_docs:
        text = text_to_words(doc, stop_words)
        total_dl += len(text)
        
    for n_text, doc in enumerate(list_of_docs):
        text = text_to_words(doc, stop_words)
        for word in query:
            if word not in text or word in stop_words:
                pass
            else:
                score = score_BM25(len(index_dict[word]), frequency[word][str(n_text)]/len(text), len(list_of_docs), 
                                   len(text), (total_dl/len(list_of_docs)))
                if str(n_text) in BM25_for_all:
                    BM25_for_all[str(n_text)] = BM25_for_all[str(n_text)] + score
                else:
                    BM25_for_all[str(n_text)] = score
        
    if  BM25_for_all == {}:
        print('Ничего не найдено')
        
    return BM25_for_all

In [4]:
m = Mystem()
def text_to_words(filename, stop_words):

    all_words = m.lemmatize(open(filename, 'r', encoding='UTF-8').read())

    all_words_clean = []
    for w in all_words:
        if w in stop_words or re.search('[А-Яа-я]', w) == None:
            pass
        else:
            all_words_clean.append(w)
    
    return all_words_clean


In [5]:

def reverse_index(list_of_docs, stop_words, index_dict):
    #функция выдающая массив индексов для каждого слова в коллекции документов
    for n_text, doc in tqdm.tqdm_notebook(enumerate(list_of_docs)):
        text = text_to_words(doc, stop_words)
        for word in set(text):
            index_dict.setdefault(word, [])
            if str(n_text) not in index_dict[word]:
                index_dict[word].append(str(n_text))
            
       
    with open('reverse_index.txt', 'w',encoding='utf-8') as outfile:
        json.dump(index_dict, outfile, ensure_ascii = False)



In [6]:
def word_frequency(list_of_docs, stop_words, frequency):            
    #функция, подсчитывающая частоту каждого слова для каждого документа в коллекции
    for n, doc in tqdm.tqdm_notebook(enumerate(list_of_docs)):
        text = text_to_words(doc, stop_words)
        for word in text:
            if word in frequency_dict:
                if str(n) in frequency[word]:
                    frequency[word][str(n)] += 1
                else:
                    frequency[word][str(n)] = 1
            else:
                frequency[word] = {}
                frequency[word][str(n)] = 1    
    
    with open('word_frequency.txt', 'w',encoding='utf-8') as outfile:
        json.dump(frequency, outfile, ensure_ascii = False)
        
            

In [10]:
k1 = 2.0
b = 0.75
stop_words_list = ["а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видеть","вместе","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делать","дело","день","деньги","десятый","десять","для","до","довольно","долго","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать","значит","значить","и","идти","из","или","им","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могут","мож","может","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я"]
q = input("search: ")
index_dict = {}
frequency_dict = {}
list_of_docs = []
BM25_for_all = {}
m = Mystem()
query = m.lemmatize(q)
for filename in tqdm.tqdm_notebook(glob.iglob('./articles/*.txt')):
    list_of_docs.append(filename)
#list_of_docs = []    
#for i in range(1000):
    #list_of_docs.append(whole_list_of_docs[i])
    

#word_frequency(list_of_docs, stop_words_list, frequency_dict)
#reverse_index(list_of_docs, stop_words_list, index_dict)

    


search: депутат


A Jupyter Widget

In [11]:
with open('word_frequency.txt') as f:    
    frequency = json.load(f)
    
with open('reverse_index.txt') as f: 
    index_dict = json.load(f) 

BM25 = BM25_foralldocs(query, list_of_docs, frequency, index_dict, BM25_for_all, stop_words_list)

In [23]:
list_of_scores = sorted(BM25.items(), key=operator.itemgetter(1), reverse = True)

for i in range(10):
    if len(tul) >= i+1:
        tul = list_of_scores[i]
        with open(list_of_docs[int(tul[0])],'r',encoding='utf-8') as f:
            print(str((re.findall('@url\t(.*)', f.read()))[0]))
    else:
        pass
        

http://www.belrab.ru/rubriki/politika/item/2135-deputat-pomogaet-detyam
http://www.belrab.ru/rubriki/bud-v-kurse/item/1123-vladimir-putin-nagradil-zuguru-rakhmatullinu
http://www.belrab.ru/rubriki/politika/item/958-deputaty-zavershili-svoyu-rabotu
http://www.belrab.ru/rubriki/politika/item/1220-deputaty-sdelali-vybor
http://www.belrab.ru/rubriki/politika/item/1491-nachalas-podgotovka-k-munitsipalnym-vyboram
http://www.belrab.ru/rubriki/bud-v-kurse/item/1087-ofitsialnye-itogi
http://www.belrab.ru/rubriki/politika/item/478-kak-vybirali-deputatov-gosdumy
http://www.belrab.ru/rubriki/politika/item/2072-nado-idti-na-vybory
http://www.belrab.ru/rubriki/politika/item/1793-sdelat-pravilnyj-vybor
http://www.belrab.ru/rubriki/politika/item/1063-ozvucheny-predvaritelnye-itogi
